# Geodesics of $E(2) \ / \ \Gamma$ for different wallpaper groups $\Gamma$

Let $E(2)$ be the group of orientation-preserving isometries on the Euclidean plane $E^2$. In this notebook, we study periodic tilings of $E^2$ by nonoverlapping shapes with no gaps. For a tiling of $E^2$, we define its symmetry group $\Gamma\subseteq E(2)$ to be the subgroup of isometries that map the tiling to itself. 

The **point group** of $\Gamma$ is $\Gamma_0 = \{\rho_{\theta}\ | \ (a,\theta)\in \Gamma \text{ for some } a \in \mathbb{R}^2\}$

The **lattice** of $\Gamma$ is $\Gamma_L = \{a\in \mathbb{R}^2\ | \ (a,0)\in \Gamma\}$ 

If $\Gamma$ is the symmetry group of a tiling, one can show that:
1. $\Gamma_0\in \{C_1,C_2,C_3,C_4,C_6\}$ where $C_n = \{\rho_\theta \ | \ \theta = \frac{i \cdot 2 \pi}{n} \text{ for } i = 0,1,2,\ldots, n-1\}$ (we refer to these as the five wallpaper groups).
2. There exists linearly independent $a,b\in \mathbb{R}^2$ such that $\Gamma_L=\{ma+nb \ | \ m,n\in \mathbb{Z}\}$ (we refer to $a$ and $b$ as the lattice generators of $\Gamma$).

For a tiling with symmetry group $\Gamma$, we can build 3-dimensional parameterization of $E(2)\ / \ \Gamma$ that depends on the wallpaper group and the lattice generators. This notebook explores how to define distance on these parameterizations and how this can be used to study the homology of the parameterized space.

In [1]:
# importing needed libraries
import numpy as np
from numpy import pi
from math import dist
import matplotlib.pyplot as plt
from ripser import ripser
from persim import plot_diagrams
# For image and video manipulation
from PIL import Image
from PIL import ImageFilter
from IPython.display import display
import glob

In [2]:
# defining the distance function described above
def C1_wrapper(a,b):
    def distance_C1(x, y):
        Y = []
        options = [-1,0,1]
        for option_a in options:
            for option_b in options:
                for option_c in options:
                    Y.append([y[0] + a[0] * option_a + b[0] * option_b, y[1] + a[1] * option_a + b[1] * option_b, y[2] + 2*pi * option_c])

        distances = []
        for y_new in Y: 
            distances.append(dist(x,y_new))

        return min(distances)
    return distance_C1

In [3]:
def generate_c1_isoms(n_isos, a, b):
    v0 = (0,0)
    v1 = a
    v2 = b
    
    rand1 = list(np.random.random(n_isos))
    rand2 = list(np.random.random(n_isos))
    
    theta = list(2*pi * np.random.random(n_isos)) 

    parallelogram = []
    
    for i in range(n_isos):
        X = rand1[i] * (v1[0]-v0[0]) + rand2[i] * (v2[0]-v0[0])
        Y = rand1[i] * (v1[1]-v0[1]) + rand2[i] * (v2[1]-v0[1])
        parallelogram.append((X,Y,theta[i]))
        
    return parallelogram

In [4]:
def C2_wrapper(a,b):
    def distance_C2(x, y):

        # no rotations
        Y = []
        options = [-1,0,1]
        for option_a in options:
            for option_b in options:
                Y.append([y[0] + a[0] * option_a + b[0] * option_b, y[1] + a[1] * option_a + b[1] * option_b, y[2]])

        # rotate by pi
        options = [-1,0,1]
        for option_a in options:
            for option_b in options:
                Y.append([a[0] + b[0] - y[0] + a[0] * option_a + b[0] * option_b, a[1] + b[1] - y[1] + a[1] * option_a + b[1] * option_b, y[2] + pi])

        # rotate by -pi
        options = [-1,0,1]
        for option_a in options:
            for option_b in options:
                Y.append([a[0] + b[0] - y[0] + a[0] * option_a + b[0] * option_b, a[1] + b[1] - y[1] + + a[1] * option_a + b[1] * option_b, y[2] - pi])

        distances = []
        for y_new in Y: 
            distances.append(dist(x,y_new))

        return min(distances)
    return distance_C2

In [5]:
def generate_c2_isoms(n_isos, a, b):
    v0 = (0,0)
    v1 = a
    v2 = b
    
    rand1 = list(np.random.random(n_isos))
    rand2 = list(np.random.random(n_isos))
    
    theta = list(pi * np.random.random(n_isos)) 

    parallelogram = []
    
    for i in range(n_isos):
        X = rand1[i] * (v1[0]-v0[0]) + rand2[i] * (v2[0]-v0[0])
        Y = rand1[i] * (v1[1]-v0[1]) + rand2[i] * (v2[1]-v0[1])
        parallelogram.append((X,Y,theta[i]))
        
    return parallelogram

In [6]:
def C4_wrapper(a): 
    def distance_C4(x, y):

        # no rotation
        Y = []
        options = [-1,0,1]
        for option_1 in options:
            for option_2 in options:
                Y.append([y[0] + a * option_1, y[1] + a * option_2, y[2]])

        # rotate by pi/2
        options = [-1,0,1]
        for option_1 in options:
            for option_2 in options:
                Y.append([a - y[1] + a * option_1, y[0] + a * option_2, y[2] + pi/2])

        # rotate by -pi/2
        options = [-1,0,1]
        for option_1 in options:
            for option_2 in options:
                Y.append([y[1] + a * option_1, -y[0] + a + a * option_2, y[2] - pi/2])

        distances = []
        for y_new in Y: 
            distances.append(dist(x,y_new))

        return min(distances)
    return distance_C4

In [7]:
# isometries that allow rotation up to pi/2
def generate_c4_isoms(n_isos, a):
    X = list(a * np.random.random(n_isos)) 
    Y = list(a * np.random.random(n_isos)) 
    theta = list(pi/2 * np.random.random(n_isos)) 
    
    isoms = []
    for i in range(n_isos):
        isoms.append((X[i],Y[i],theta[i]))
        
    return isoms

In [8]:
def C3_wrapper(a):   
    def distance_C3(x, y):

        # no translation in rotation axis
        Y = [y] # no translation in any axis
        for i in [1,-1]:
            Y.append((y[0], y[1]+ i * (3**0.5) * a, y[2])) # front and back
            Y.append((y[0] + i * (3/2) * a, y[1] - (3**0.5)/2 * a, y[2])) # LB and RF
            Y.append((y[0] + i * (3/2) * a, y[1] + (3**0.5)/2 * a, y[2])) # LF and RB

        # rotate by 2pi/3
        Y.append((-1/2 * y[0] - (3**0.5)/2 * y[1], -1/2 * y[1] + (3**0.5)/2 * y[0], y[2] + 2*pi/3)) # only rotation by 2pi/3
        for i in [1,-1]:
            y1 = -1/2 * y[0] - (3**0.5)/2 * y[1]
            y2 = -1/2 * y[1] + (3**0.5)/2 * y[0] 
            Y.append((y1, y2 + i * (3**0.5) * a, y[2] + 2*pi/3)) # front and back
            Y.append((y1 + i * (3/2) * a, y2 - (3**0.5)/2 * a, y[2] + 2*pi/3)) # LB and RF
            Y.append((y1 + i * (3/2) * a, y2 + (3**0.5)/2 * a, y[2] + 2*pi/3)) # LF and RB    

        # rotate by -2pi/3
        Y.append((-1/2 * y[0] + (3**0.5)/2 * y[1], -1/2 * y[1] - (3**0.5)/2 * y[0], y[2] - 2*pi/3)) # only rotation by -2pi/3
        for i in [1,-1]:
            y1 = -1/2 * y[0]+ (3**0.5)/2 * y[1]
            y2 = -1/2 * y[1] - (3**0.5)/2 * y[0]
            Y.append((y1, y2 + i * (3**0.5) * a, y[2] - 2*pi/3)) # front and back
            Y.append((y1 + i * (3/2) * a, y2 - (3**0.5)/2 * a, y[2] - 2*pi/3)) # LB and RF
            Y.append((y1 + i * (3/2) * a, y2 + (3**0.5)/2 * a, y[2] - 2*pi/3)) # LF and RB   

        distances = []
        for y_new in Y: 
            distances.append(dist(x,y_new))

        return min(distances)
    return distance_C3

In [9]:
# allow only for rotations up to 2pi/3
# we first sample from a rectangular patch of the plane and then cut off the corners to create a hexagonal shape

def generate_c3_isoms(n_isos, a):
    X = list(2 * a * np.random.random(2 * n_isos) - a)  
    Y = list((3**0.5) * a * np.random.random(2 * n_isos) - (3**0.5)/2 * a)
    theta = list(2*pi/3 * np.random.random(2 * n_isos))
    
    i = 0
    hexagon = []
    while len(hexagon) < n_isos:
        if (Y[i] > -(3**0.5) * a + (3**0.5) * X[i]) and (Y[i] > -(3**0.5) * a - (3**0.5) * X[i]) and (Y[i] < (3**0.5) * a + (3**0.5) * X[i]) and (Y[i] < (3**0.5) * a - (3**0.5) * X[i]):  
            hexagon.append((X[i], Y[i], theta[i]))
        i = i + 1
    
    return hexagon

In [20]:
def C6_wrapper(a):   
    def distance_C6(x, y):

        # no translation in rotation axis
        Y = [y] # no translation in any axis
        for i in [1,-1]:
            Y.append((y[0], y[1] + i * (3**0.5) * a, y[2])) # front and back
            Y.append((y[0] + i * (3/2) * a, y[1] - (3**0.5)/2 * a, y[2])) # LB and RF
            Y.append((y[0] + i * (3/2) * a, y[1] + (3**0.5)/2 * a, y[2])) # LF and RB

        # rotation by pi/3
        Y.append((1/2 * y[0] - (3**0.5)/2 * y[1], 1/2 * y[1] + (3**0.5)/2 * y[0], y[2] + pi/3)) # only rotation by 2pi/3
        for i in [1,-1]:
            y1 = 1/2 * y[0] - (3**0.5)/2 * y[1]
            y2 = 1/2 * y[1] + (3**0.5)/2 * y[0] 
            Y.append((y1, y2 + i * (3**0.5) * a, y[2] + 2*pi/3)) # front and back
            Y.append((y1 + i * (3/2) * a, y2 - (3**0.5)/2 * a, y[2] + pi/3)) # LB and RF
            Y.append((y1 + i * (3/2) * a, y2 + (3**0.5)/2 * a, y[2] + pi/3)) # LF and RB    

        # rotation by -pi/3
        Y.append((1/2 * y[0] + (3**0.5)/2 * y[1], 1/2 * y[1] - (3**0.5)/2 * y[0], y[2] - pi/3)) # only rotation by -2pi/3
        for i in [1,-1]:
            y1 = 1/2 * y[0]+ (3**0.5)/2 * y[1]
            y2 = 1/2 * y[1] - (3**0.5)/2 * y[0]
            Y.append((y1, y2 + i * (3**0.5) * a, y[2] - 2*pi/3)) # front and back
            Y.append((y1 + i * (3/2) * a, y2 - (3**0.5)/2 * a, y[2] - pi/3)) # LB and RF
            Y.append((y1 + i * (3/2) * a, y2 + (3**0.5)/2 * a, y[2] - pi/3)) # LF and RB   

        distances = []
        for y_new in Y: 
            distances.append(dist(x,y_new))

        return min(distances)
    return distance_C6

In [11]:
# only allow rotation up to pi/3
def generate_c6_isoms(n_isos, a):
    X = list(2 * a * np.random.random(2 * n_isos) - a)  
    Y = list((3**0.5) * a * np.random.random(2 * n_isos) - (3**0.5)/2 * a)
    theta = list(pi/3 * np.random.random(2 * n_isos))
    
    i = 0
    hexagon = []
    while len(hexagon) < n_isos:
        if (Y[i] > -(3**0.5) * a + (3**0.5) * X[i]) and (Y[i] > -(3**0.5) * a - (3**0.5) * X[i]) and (Y[i] < (3**0.5) * a + (3**0.5) * X[i]) and (Y[i] < (3**0.5) * a - (3**0.5) * X[i]):  
            hexagon.append((X[i], Y[i], theta[i]))
        i = i + 1
    
    return hexagon

# Comparing ideal distances with pixel distances

Given an image, a lens size, and a translation, this function returns a snapshot after the translation is performed.

In [12]:
def translation_snapshot(img, lens_sz, translation):
    (a,b) = translation 
    translation = (-a,-b)
    (x,y) = upper_left_center(img, translation)
    (length, height) = img.size
    
    lens_corner_x = x - lens_sz/2.0 
    lens_corner_y = y - lens_sz/2.0 
    ret_img = img.crop((lens_corner_x,lens_corner_y, lens_corner_x+lens_sz, lens_corner_y+lens_sz))
    
    #make sure didn't fall off of the image
    (ret_length, ret_height) = ret_img.size
    if (lens_corner_x<0 or lens_corner_y<0 or lens_corner_x+lens_sz>length or lens_corner_y+lens_sz>height):
        print("I came off the page!")
        return False
    return ret_img

In [13]:
def upper_left_center(img, center):
    (x,y) = center
    (l, h) = img.size
    return (l/2.0+x , h/2.0 - y) 

In [14]:
def create_snapshot(img, lens_sz, isometry):
    (x,y,rotation) = isometry
    translation = (x,y)
    # turn from radian to degree
    rotation = 57.2958*rotation
    rot_img = img.rotate(rotation)
    return translation_snapshot(rot_img, lens_sz, translation)

In [15]:
 def pixel_wrapper():
    def image_euclidean_distance(img1, img2):
        arr1 = np.asarray(img1)
        arr1 = arr1.reshape(-1)
        arr2 = np.asarray(img2)
        arr2 = arr2.reshape(-1)
        return dist(arr1, arr2)
    return image_euclidean_distance

distance_lst() takes a list of points and a metric, and returns a list of all the pairwise distances between the points.

In [16]:
def distance_lst(lst, metric):
    if len(lst)==2:
        return [metric(lst[0], lst[1])]
    if len(lst)<2:
        return []
    return first_slot_lst(lst, metric) + distance_lst(lst[1:], metric)

first_slot_lst is a helper function for distance_list()

In [17]:
def first_slot_lst(lst, metric):
    ret_lst = []
    x = lst[0]
    new_list = lst[1:]
    for i in new_list:
        ret_lst.append(metric(x, i))
    return ret_lst

In [18]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

In [86]:
# Grab image from files
img = Image.open('images/C4pinwheel.png')
img = img.convert('L')
img = img.filter(ImageFilter.BLUR)

# lens size, how much to stretch fund domain by
pixel_scale = 40
a = 1
n_isoms = 500
c4_isom = generate_c4_isoms(n_isoms,a)

# scale the isometries by pixel_scale so they can be used on the image
scaled_c4_isom = c4_isom
for i in range(len(c4_isom)):
    e = c4_isom[i]
    scaled_c4_isom[i] = (pixel_scale*e[0], pixel_scale*e[1], e[2])

# create list of images taken after isometries
c4_isom_images = []
for isom in scaled_c4_isom:
    c4_isom_images.append(create_snapshot(img, pixel_scale, isom))

# place all the pairwise distances in arrays and normalize them for comparison
C4_ideal_func = C4_wrapper(a)
C4_pixel_func = pixel_wrapper()
c4_ideal_dist_lst = distance_lst(c4_isom, C4_ideal_func) 
c4_pixel_dist_lst = distance_lst(c4_isom_images, C4_pixel_func) 
c4_ideal_dist_lst = normalize(c4_ideal_dist_lst)
c4_pixel_dist_lst = normalize(c4_pixel_dist_lst)

dist(c4_ideal_dist_lst,c4_pixel_dist_lst)

0.46571541714459497

In [21]:
# Grab image from files
img = Image.open('images/C6 star.png')
img = img.convert('L')
img = img.filter(ImageFilter.BLUR)

# lens size, how much to stretch fund domain by
pixel_scale = 60
a = 1
n_isoms = 500
c6_isom = generate_c6_isoms(n_isoms,a)

# scale the isometries by pixel_scale so they can be used on the image
scaled_c6_isom = c6_isom
for i in range(len(c6_isom)):
    e = c6_isom[i]
    scaled_c6_isom[i] = (pixel_scale*e[0], pixel_scale*e[1], e[2])

# create list of images taken after isometries
c6_isom_images = []
for isom in scaled_c6_isom:
    c6_isom_images.append(create_snapshot(img, pixel_scale, isom))

# place all the pairwise distances in arrays and normalize them for comparison
C6_ideal_func = C6_wrapper(a)
C6_pixel_func = pixel_wrapper()
c6_ideal_dist_lst = distance_lst(c6_isom, C6_ideal_func) 
c6_pixel_dist_lst = distance_lst(c6_isom_images, C6_pixel_func) 
c6_ideal_dist_lst = normalize(c6_ideal_dist_lst)
c6_pixel_dist_lst = normalize(c6_pixel_dist_lst)

dist(c6_ideal_dist_lst,c6_pixel_dist_lst)

0.5456013502386748

In [23]:
# Grab image from files
img = Image.open('images/C3gradient.png')
img = img.convert('L')
img = img.filter(ImageFilter.BLUR)

# lens size, how much to stretch fund domain by
pixel_scale = 65
a = 1
n_isoms = 500
c3_isom = generate_c3_isoms(n_isoms,a)

# scale the isometries by pixel_scale so they can be used on the image
scaled_c3_isom = c3_isom
for i in range(len(c3_isom)):
    e = c3_isom[i]
    scaled_c3_isom[i] = (pixel_scale*e[0], pixel_scale*e[1], e[2])

# create list of images taken after isometries
c3_isom_images = []
for isom in scaled_c3_isom:
    c3_isom_images.append(create_snapshot(img, pixel_scale, isom))

# place all the pairwise distances in arrays and normalize them for comparison
C3_ideal_func = C3_wrapper(a)
C3_pixel_func = pixel_wrapper()
c3_ideal_dist_lst = distance_lst(c3_isom, C3_ideal_func) 
c3_pixel_dist_lst = distance_lst(c3_isom_images, C3_pixel_func) 
c3_ideal_dist_lst = normalize(c3_ideal_dist_lst)
c3_pixel_dist_lst = normalize(c3_pixel_dist_lst)

dist(c3_ideal_dist_lst,c3_pixel_dist_lst)

0.458969742635637

In [26]:
# Grab image from files
img = Image.open('images/C2bow.png')
img = img.convert('L')
img = img.filter(ImageFilter.BLUR)

# lens size, how much to stretch fund domain by
pixel_scale = 50
a = (0,-2)
b =(2,1)
n_isoms = 500
c2_isom = generate_c2_isoms(n_isoms,a, b)

# scale the isometries by pixel_scale so they can be used on the image
scaled_c2_isom = c2_isom
for i in range(len(c2_isom)):
    e = c2_isom[i]
    scaled_c2_isom[i] = (pixel_scale*e[0], pixel_scale*e[1], e[2])

# create list of images taken after isometries
c2_isom_images = []
for isom in scaled_c2_isom:
    c2_isom_images.append(create_snapshot(img, pixel_scale, isom))

# place all the pairwise distances in arrays and normalize them for comparison
C2_ideal_func = C2_wrapper(a,b)
C2_pixel_func = pixel_wrapper()
c2_ideal_dist_lst = distance_lst(c2_isom, C2_ideal_func) 
c2_pixel_dist_lst = distance_lst(c2_isom_images, C2_pixel_func) 
c2_ideal_dist_lst = normalize(c2_ideal_dist_lst)
c2_pixel_dist_lst = normalize(c2_pixel_dist_lst)

dist(c2_ideal_dist_lst,c2_pixel_dist_lst)

0.47334755752503926

In [27]:
img = Image.open('images/C1music.png')
img = img.convert('L')
img = img.filter(ImageFilter.BLUR)
img.show()

In [28]:
# Grab image from files
img = Image.open('images/C1music.png')
img = img.convert('L')
img = img.filter(ImageFilter.BLUR)

# lens size, how much to stretch fund domain by
pixel_scale = 35
a = (6,1)
b =(-1,6)
n_isoms = 500
c1_isom = generate_c1_isoms(n_isoms,a, b)

# scale the isometries by pixel_scale so they can be used on the image
scaled_c1_isom = c1_isom
for i in range(len(c1_isom)):
    e = c1_isom[i]
    scaled_c1_isom[i] = (pixel_scale*e[0], pixel_scale*e[1], e[2])

# create list of images taken after isometries
c1_isom_images = []
for isom in scaled_c1_isom:
    c1_isom_images.append(create_snapshot(img, pixel_scale, isom))

# place all the pairwise distances in arrays and normalize them for comparison
C1_ideal_func = C1_wrapper(a,b)
C1_pixel_func = pixel_wrapper()
c1_ideal_dist_lst = distance_lst(c1_isom, C1_ideal_func) 
c1_pixel_dist_lst = distance_lst(c1_isom_images, C1_pixel_func) 
c1_ideal_dist_lst = normalize(c1_ideal_dist_lst)
c1_pixel_dist_lst = normalize(c1_pixel_dist_lst)

dist(c1_ideal_dist_lst,c1_pixel_dist_lst)

0.48091055597818766